<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# BetUnfair™


---

### Aim

- Make bare $$$

- Robin hood the crap out of betting companies

<h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc"><ul class="toc-item"><li><span><a href="#FIFA-Ultimate-Team-Ratings">FIFA Ultimate Team Ratings</a></span>

In [ ]:
# example markdown for a sub-bullet point        
# <ul class="toc-item"><li><span><a href="#Example-title">Example title</a></span></li>

In [145]:
import requests

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from pprint import pprint
import numpy as np
import re
import patsy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as sm
from statsmodels.tsa.arima_model import ARIMA
import warnings
import zipfile

import os
import sys
import json
import webbrowser
from json.decoder import JSONDecodeError
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, AdaBoostRegressor, \
GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn import svm
from sklearn.tree import export_graphviz
from IPython import display

from joblib import dump, load

from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette(sns.diverging_palette(130, 260, center='light'),3))
import datetime
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import bokeh
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

from scipy.sparse import coo_matrix, hstack
from scipy import stats
from scipy import signal

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set(font_scale=1.5)



warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

## FIFA Ultimate Team Ratings

Below code scrapes all the FUT ratings from the beginning of the 2018-19 season from [here](https://www.fifaindex.com/players/fifa19_279/1/?gender=male&order=desc).

Included are the name, nationality, beginning of the season rating (rather than [in-forms](https://www.easports.com/uk/fifa/ultimate-team/fut/team/19) and [winter upgrades](https://www.ea.com/games/fifa/fifa-19/news/fut-winter-upgrades-top-five?isLocalized=true)), top 3 positions (where applicable), age and team as of Summer 2018.

In [131]:
# Scrapes every player in FIFA's database. N.B. players can have 1, 2, or 3 preferred positions -
# for the sake of consistency we put 3 columns of positions, with NaNs if they have fewer
players = []

counter = 1
params = {
    'gender' : 'male',
    'order' : 'desc'
}

r_ = requests.get('https://www.fifaindex.com/players/fifa19_279/1/', params=params)
soup = BeautifulSoup(r_.text, 'html.parser')

while soup.findAll('tr'):

    for player in tqdm_notebook(soup.findAll('tr')[1:]):
        footballer = []

        try:
            footballer.append(player.find('td', attrs={'data-title': 'Name'}).text)
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'Nationality'}).find('a')['title'])
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'OVR / POT'}).find('span', attrs = {'class': True}).text)
        except:
            footballer.append(np.nan)

        positions =  player.find('td', attrs={'data-title': 'Preferred Positions'}).find_all('a')

        for i in range(3):

            try:
                footballer.append(positions[i]['title'])
            except:
                footballer.append(np.nan)

        try:
            footballer.append(player.find('td', attrs={'data-title': 'Age'}).text)
        except:
            footballer.append(np.nan)
        try:
            footballer.append(player.find('td', attrs={'data-title': 'Team'}).find('a')['title'])
        except:
            footballer.append(np.nan)

        players.append(footballer)
    counter += 1
    r_ = requests.get(f'https://www.fifaindex.com/players/fifa19_279/{counter}/', params=params)
    soup = BeautifulSoup(r_.text, 'html.parser')

In [133]:
cols = [
    'name',
    'nationality',
    'rating',
    'position1',
    'position2',
    'position3',
    'age',
    'team'
    
]

playaz = pd.DataFrame(players, columns=cols)

In [147]:
# If it's a big CSV add the argument compression='zip'
playaz.to_csv('playaz.csv', index=False)

#### For reference, this is how to zip files in Python

```python
with zipfile.ZipFile('pin_bet365_games.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write('pin_bet365_games.csv')
```

In [159]:
# And unzip them (yes I know this is dutty data let's sort it tomorrow)
with zipfile.ZipFile('pin_bet365_games.zip', 'r') as zf:
    with zf.open('pin_bet365_games.csv') as myZip:
        bet365 = pd.read_csv(myZip)